<a href="https://colab.research.google.com/github/JCGOMEZSALA/JCGOMEZSALA.github.io/blob/master/Interactive_Notebooks_mio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://web.ua.es/es/defc/imagenes/logotipo-defc.png" width="180">

# Interactive Notebooks

-  [Ejemplos](https://www.stratascratch.com/python.html)

## Importing the modules

**ipywidgets** is a new module we we will learn about in this lesson

In [0]:
!pip install psycopg2

In [0]:
!pip install ipywidgets

In [0]:
import os
import pandas as pd
import numpy  as np

# import psycopg2 as ps
import matplotlib.pyplot as plt
import ipywidgets as widgets

# back end of ipywidgets
from IPython.display import display

## Getting the data

We will use **dc_bikeshare_q1_2012** dataset

In [0]:
'''
# Connect to database
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
port = '5432'
# TODO Write your username and database password in the following two variables
user_name = '' #enter username
pwd = '' #enter your database password found in the profile tab in Strata Scratch

try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

'''

"\n# Connect to database\nhost_name = 'db-strata.stratascratch.com'\ndbname = 'db_strata'\nport = '5432'\n# TODO Write your username and database password in the following two variables\nuser_name = '' #enter username\npwd = '' #enter your database password found in the profile tab in Strata Scratch\n\ntry:\n    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)\nexcept ps.OperationalError as e:\n    raise e\nelse:\n    print('Connected!')\n\n"

In [0]:
'''
def get_dataset(dataset_name):
    #Write SQL below to pull datasets 
    cur = conn.cursor()
    cur.execute("""SELECT *  FROM datasets.{0};""".format(dataset_name))
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description] 
    conn.commit()

    #create the pandas dataframe
    dataframe = pd.DataFrame(data, columns=colnames)

    #close the connection
    cur.close()
    
    return dataframe

dc_bikeshare = get_dataset("dc_bikeshare_q1_2012")

'''

In [0]:
# Importacion libreria !drive
from google.colab import drive

# Peticion e introduccion de clave
drive.mount('/content/drive')

In [0]:
# Importacion de datos situados en una determinada carpeta
dc_bikeshare = pd.read_csv('drive/My Drive/COLAB_NOTEBOOKS/Manual_de_Google/dc_bikeshare_q1_2012.csv')

In [0]:
dc_bikeshare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361317 entries, 0 to 361316
Data columns (total 9 columns):
Duration                361317 non-null int64
Start date              361317 non-null object
End date                361317 non-null object
Start station number    361317 non-null int64
Start station           361317 non-null object
End station number      361317 non-null int64
End station             361317 non-null object
Bike number             361317 non-null object
Member type             361317 non-null object
dtypes: int64(3), object(6)
memory usage: 24.8+ MB


## Introduction to widgets

### What are widgets

- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html

Widgets are everything you see on screen when using any application and it is controlable by you. For example a button is widget because you can click it, a menu is a widget because you can select some item from that menu and so on. Each widget is a way for the user to control some value behind the widget. Son in essence widgets are user controlable function inputs.

There is also another class of widgets called layouts. In ipywidgets we have HBox, VBox, Accordion and Tabs.

Finally for convenience purpose we also non controllable widgets like labels and tooltips which are just text.

### Widget types

- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

Jupyter supports sliders, range sliders, int and float text inputs, check boxes, drop downs, radio buttons, selectors, play widgets, date time selectors, color selectors and many more.

### Creating widgets
* To create any widget we use class constructors for that widget (e.g. `my_widget = widgets.IntSlider()`).
* To display the widget we use the `display` function.
* To get the value of the widget in some cell later we use `my_widget.value`
* To release a widget call its close method (e.g. `my_widget.close()`)

In [0]:
w = widgets.IntSlider()

display(w)

IntSlider(value=0)

In [0]:
w.value * 42

0

###  The update problem

You will quickly notice that the cell with code `w.value * 42` does not recompute itself automatically when you change the value of the widget and you must execute the cell manually to get new results. The reason for this behaviour is that these two cells are not aware of each other's changes.

To make widgets which are aware of their environment and that we want some code to be executed whenever a widget changes value we use the `widgets.interact` method.

- https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

In [0]:
def count_of_rides(duration_seconds_upper_bound):
    output = dc_bikeshare[dc_bikeshare.duration_seconds <= duration_seconds_upper_bound].shape[0]
    
    return "There were a total of {0} rides shorter than {1} seconds"\
           .format(output, duration_seconds_upper_bound)

count_of_rides(120.0)

'There were a total of 5786 rides shorter than 120.0 seconds'

In [0]:
widgets.interact(count_of_rides, 
                 duration_seconds_upper_bound = (0.0, 600.0, 1.0))

<function __main__.count_of_rides>

To use interact we need two things.
- Function which will provide some output given an input provided by widgets
- Description of input widgets

To describe a input widgets we can use one of two formats:
- Short (abbreviated) format (e.g. for a float slider (0.0, 600.0, 1.0))
- Long format (e.g. for a slider widgets.FloatSlider(min=0.0, max=600.0, step=1.0)


- https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Widget-abbreviations

The description of input widgets must be precise in the sense that if we define `f(x, y, z)` we must use `interact(f, x=..., y=..., z=...)` that is the names of the parameters must match in both the function and in `interact`.

In [0]:
def count_of_rides_between(start_seconds, end_seconds):
    query_str = "duration_seconds >= {0} and duration_seconds <= {1}".format(start_seconds, end_seconds)
    
    output = dc_bikeshare.query(query_str).shape[0]
    
    return "There were a total of {0} rides between {1} and {2} seconds"\
           .format(output, start_seconds, end_seconds)
    
count_of_rides_between(60.0, 120.0)

'There were a total of 2575 rides between 60.0 and 120.0 seconds'

In [0]:
widgets.interact(count_of_rides_between,
                 start_seconds = (0.0, 600.0, 1.0),
                 end_seconds = widgets.FloatSlider(min=0.0, max=600.0, step=1.0))

<function __main__.count_of_rides_between>

The example above demonstrates using two widgets and a function with two parameters. There is no bound on the number of widgets and parameters. The next example will show how to use a function with 3 parameters.

In [0]:
def controlled_filter(start_station, bike_number, rider_type):
    mask1 = dc_bikeshare.start_station == start_station
    mask2 = dc_bikeshare.bike_number == bike_number
    mask3 = dc_bikeshare.rider_type == rider_type
    
    data = dc_bikeshare[mask1 & mask2 & mask3]
    
    return data.head()

In [0]:
start_stations = dc_bikeshare.start_station.unique()
bike_numbers   = dc_bikeshare.bike_number.unique()
rider_types    = dc_bikeshare.rider_type.unique()

# we use short formats to define widgets here
widgets.interact(controlled_filter,
         start_station = start_stations.tolist(),
         bike_number = bike_numbers.tolist(),
         rider_type = rider_types.tolist()
        )

<function __main__.controlled_filter>

As you just saw your function can return anything including dataframes. Actually we can also make dynamic plots and the following example will demonstrate that.

The plots will be histograms of duration for each starting station.

In [0]:
def plot_histogram(start_station, color):
    data_local = dc_bikeshare[dc_bikeshare.start_station == start_station].copy()
    
    plt.hist(data_local.duration_seconds, range=(0, 1800), bins=200, color=color, alpha=0.8)
    # gca stands for get current axis
    plt.gca().set_title("Duration histogram for {0}".format(start_station))
    
    plt.gca().set_xlabel("Duration in seconds")
    plt.gca().set_ylabel("Count")
    
    plt.show()

start_stations = dc_bikeshare.start_station.unique()

widgets.interact(plot_histogram, 
                 start_station = start_stations.tolist(), 
                 color=["green", "orange", "xkcd:navy blue"])

<function __main__.plot_histogram>

Finally we will see the Play Widget in use.

We will redo our histogram example but now instead of users manually selecting streets an animation going through all streets will be played.

Note that the play widgets iterates over a range of integers and gives integers so we will need to use integer position indexing.

You may need to explicitely press the play button.

In [0]:
start_stations = dc_bikeshare.start_station.unique()

def plot_histogram_animated(start_station_idx, color):
    start_station = start_stations[start_station_idx]
    
    data_local = dc_bikeshare[dc_bikeshare.start_station == start_station].copy()
    
    plt.hist(data_local.duration_seconds, range=(0, 1800), bins=200, color=color, alpha=0.8)
    # gca stands for get current axis
    plt.gca().set_title("Duration histogram for {0}".format(start_station))
    
    plt.gca().set_xlabel("Duration in seconds")
    plt.gca().set_ylabel("Count")
    
    plt.show()
    
widgets.interact(plot_histogram_animated, 
                 start_station_idx = widgets.Play(min=0, max=start_stations.shape[0], step=1, value=0), 
                 color=["green", "orange", "xkcd:navy blue"])

<function __main__.plot_histogram_animated>

Remember there are many widgets and you can see the list of them at
* https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

### The interact_manual method

- https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#interact_manual

Sometimes the function you call with interact can take quite a while to finish and because of the nature of interact any change in the value of the widgets will call that slow function again brining your system to a crawl. 

To solve this problem `interact_manual` was invented. Using it is same as using interact except that now you get a button which you need to click for changes to take effect.

In [0]:
def slow_function(rider_type):
    # For the sake of example assume this was a slow function
    out = dc_bikeshare[dc_bikeshare.rider_type == rider_type].duration_seconds.median()
    
    return 'Median riding duration in seconds is ' + str(out)

widgets.interact_manual(slow_function, rider_type=dc_bikeshare.rider_type.unique())

<function __main__.slow_function>

## DATOS

-[Capital Bikeshare](https://s3.amazonaws.com/capitalbikeshare-data/index.html)

### Exercises

#### Exercise #1

There is a date picker widget in ipywidgets. Use it to make a histogram plot of durations for all rides made in the chosen date.

Hint: Use `dc_bikeshare.start_time.dt` to filter your data.

#### Exercise #2

Make an interactive count of number of times a single bike was used.

Hint: Use the bike_number column.

#### Exercise #3

Make a dataframe filter based on start_time and end_time

Hint: Ignore the time component and use date pickers.